In [1]:
import pandas as pd
import utm
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import h2o
from h2o.automl import H2OAutoML
from utils.from_latlon import from_latlon
from utils.preprocessing import preprocess
from utils.postprocessing import postprocess_data, location_averaging, calculate_error

seed = 38 # Seed for train/val split

In [2]:
# Specify data paths
train_data = 'Example_data\Output\Train_test_data\Training_Tag_GPS_locations.xlsx'
test_data = 'Example_data\Output\Train_test_data\Testing_Tag_GPS_locations.xlsx'
radio_tower_xy_path = 'Example_data\Input\Radio_tower_locations\RTEastNorth.xlsx'

# Variable parameters
freq = '5min' # Frequency of data

# Fixed parameters
routine = 'training'
data_type = ['Simulated BTFS', 'BTFS'] # Simulation or Live BTF, or could do both
dimensions = ['xOffset', 'yOffset']
predictors = ['ant1_mean', 'ant2_mean', 'ant3_mean', 'ant4_mean', 'ant1_count', 'ant2_count', 'ant3_count', 'ant4_count', 'ant1_std', 'ant2_std', 'ant3_std', 'ant4_std', 'mean_std', 'total_count']

In [3]:
def preprocess_sim_data(sim_data, data_type, freq, tower_locs, routine):
    # Get data
    sim_dat_filt = sim_data[sim_data['Data_type'].isin(data_type)]
    
    sim_dat_filt = preprocess(sim_dat_filt, freq, routine)
     
    # Calculate easting and northing from lat long
    sim_dat_filt['easting'], sim_dat_filt['northing'], sim_dat_filt['zone_num'], sim_dat_filt['zone_letter'] = from_latlon(sim_dat_filt['POINT_Y'].values, sim_dat_filt['POINT_X'].values)

    # Create a dictionary of the coordinates of the towers
    offset_dict = tower_locs.set_index('TowerID').to_dict()
    point_x = offset_dict['POINT_X']
    point_y = offset_dict['POINT_Y']

    # Standardise the coordinates so that the tower location == 0 on both the x and y axes.
    sim_dat_filt['xOffset'] = sim_dat_filt['easting'] - sim_dat_filt['TowerID'].map(point_x).fillna(0)
    sim_dat_filt['yOffset'] = sim_dat_filt['northing'] - sim_dat_filt['TowerID'].map(point_y).fillna(0)
    
    return sim_dat_filt


In [4]:
# #Convert locations predictions back to easting northings

# def postprocess_data(prediction_data, tower_locs):
#     # Create a dictionary of the coordinates of the towers
#     offset_dict = tower_locs.set_index('TowerID').to_dict()
#     point_x = offset_dict['POINT_X']
#     point_y = offset_dict['POINT_Y']

#     # Change predicted x/y offset values to their respective easting/northing considering the location of the tower
#     prediction_data['easting_pred'] = prediction_data['xOffset_pred'] + prediction_data['TowerID'].map(point_x).fillna(0)
#     prediction_data['northing_pred'] = prediction_data['yOffset_pred'] + prediction_data['TowerID'].map(point_y).fillna(0)
    
#     return prediction_data

In [5]:
# Get training data
train_data = pd.read_excel(train_data)
train_data['DateAndTime'] = pd.to_datetime(train_data['DateAndTime'])

# Get testing data
test_data = pd.read_excel(test_data)
test_data['DateAndTime'] = pd.to_datetime(test_data['DateAndTime'])

# Get tower locations
tower_locs = pd.read_excel(radio_tower_xy_path)

In [6]:
# Preprocess the training and testing data
train_data_preproc = preprocess_sim_data(train_data, data_type, freq, tower_locs, routine)
test_data_preproc = preprocess_sim_data(test_data, data_type, freq, tower_locs, routine)

# Initialise h2o
h2o.init()

# Train, save and test the models for each dimension
for dimension in dimensions:
    print(f"Training model for {dimension}")
    # Train the model
    variables = predictors + [dimension]
    training_input = train_data_preproc[variables]
    train = h2o.H2OFrame(training_input)
    aml = H2OAutoML(max_models=20, seed=seed, stopping_metric='MAE', sort_metric='MAE')
    aml.train(x=predictors, y=dimension, training_frame=train)
    print(aml.leaderboard)

    # Save the leader model
    h2o.save_model(aml.leader, path = f"Example_data\Output\Trained_models\{dimension}", force=True)

    # Make predictions on the test data
    test = h2o.H2OFrame(test_data_preproc)
    preds = aml.leader.predict(test)

    # Save predictions to a new column in the test dataframe
    pred_column_name = f"{dimension}_pred"
    test_data_preproc[pred_column_name] = preds.as_data_frame()

# Stop h2o
h2o.cluster().shutdown()


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM Zulu17.36+17-CA (build 17.0.4.1+1-LTS, mixed mode, sharing)
  Starting server from C:\Users\s5236256\Documents\GitHub\ml4rt\.venv\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\s5236256\AppData\Local\Temp\tmppbt96s7s
  JVM stdout: C:\Users\s5236256\AppData\Local\Temp\tmppbt96s7s\h2o_s5236256_started_from_python.out
  JVM stderr: C:\Users\s5236256\AppData\Local\Temp\tmppbt96s7s\h2o_s5236256_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Australia/Brisbane
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.4
H2O_cluster_version_age:,1 month and 6 days
H2O_cluster_name:,H2O_from_python_s5236256_sfvuor
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.922 Gb
H2O_cluster_total_cores:,0
H2O_cluster_allowed_cores:,0
H2O_cluster_status:,"locked, healthy"


Training model for xOffset
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
21:25:37.13: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%
model_id                                                     mae     rmse     mse    rmsle    mean_residual_deviance
DeepLearning_grid_3_AutoML_1_20230603_212536_model_1     226.517  360.384  129877      nan                    129877
DeepLearning_grid_2_AutoML_1_20230603_212536_model_1     227.591  358.044  128196      nan                    128196
DeepLearning_grid_1_AutoML_1_20230603_212536_model_1     227.592  364.462  132833      nan                    132833
GBM_3_AutoML_1_20230603_212536                           228.134  356.116  126818      nan                    126818
GBM_grid_1_AutoML_1_20230603_212536_model_4              228.28   355.354  126276      nan                    126276
GBM_2_AutoML_1_2023

In [7]:
test_predictions = postprocess_data(test_data_preproc, tower_locs)
test_predictions

,DateTime,TowerID,TagID,Data_type,POINT_X,POINT_Y,ant1_count,ant2_count,ant3_count,ant4_count,...,northing,zone_num,zone_letter,xOffset,yOffset,xOffset_pred,yOffset_pred,easting_pred,northing_pred,zone_number
0,2021-02-02 07:50:00,RT01,60,BTFS,146.256751,-21.919946,0.0,4.0,0.0,4.0,...,7.575848e+06,55,k,-102.068451,-70.508028,-303.194767,-38.372591,423035.536417,7.575880e+06,55
1,2021-02-02 07:50:00,RT04,60,BTFS,146.256751,-21.919946,2.0,0.0,0.0,0.0,...,7.575848e+06,55,k,290.544872,174.056007,-122.810885,231.032184,422823.306976,7.575905e+06,55
2,2021-02-02 07:55:00,RT04,60,BTFS,146.255721,-21.919832,4.0,1.0,2.0,3.0,...,7.575860e+06,55,k,184.100128,186.159296,-40.587162,93.638011,422905.530699,7.575768e+06,55
3,2021-02-02 08:00:00,RT04,60,BTFS,146.255721,-21.919832,1.0,2.0,1.0,1.0,...,7.575860e+06,55,k,184.100128,186.159296,-38.753592,108.618994,422907.364269,7.575783e+06,55
4,2021-02-04 16:45:00,RT12,49,BTFS,146.295371,-22.016526,0.0,0.0,1.0,0.0,...,7.565176e+06,55,k,30.027954,-471.955543,-131.587943,-310.217039,427113.086597,7.565338e+06,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,2021-09-18 18:15:00,RT26,92,BTFS,146.389023,-22.144377,4.0,0.0,0.0,0.0,...,7.551066e+06,55,k,336.750637,-248.147779,-204.398000,275.019540,436456.464758,7.551589e+06,55
277,2021-09-18 18:15:00,RT27,92,BTFS,146.389023,-22.144377,5.0,0.0,0.0,4.0,...,7.551066e+06,55,k,21.144763,-287.590792,-167.372011,184.687492,436809.096621,7.551538e+06,55
278,2021-09-18 18:20:00,RT27,92,BTFS,146.389023,-22.144377,6.0,0.0,0.0,1.0,...,7.551066e+06,55,k,21.144763,-287.590792,-167.781096,147.803092,436808.687536,7.551501e+06,55
279,2021-09-18 18:25:00,RT26,95,BTFS,146.387478,-22.144688,0.0,0.0,1.0,0.0,...,7.551031e+06,55,k,177.568547,-283.214376,-117.054900,-321.258446,436543.807858,7.550993e+06,55


In [9]:
# Post process the test predictions to calculate location from the radio tower locations
test_predictions_tower = postprocess_data(test_data_preproc, tower_locs)

# Location averaging functions
test_location_estimates = location_averaging(test_predictions_tower)
test_location_estimates = calculate_error(test_location_estimates)

# Calculate the mean absolute error of UTM_predictions['distance'] and the standard error
mean_error = np.mean(test_location_estimates['error_m'])
std_error = stats.sem(test_location_estimates['error_m'])

print(f'Mean error (+/-SE) = {mean_error} (+/- {std_error})')

Mean error (+/-SE) = 266.43968231942773 (+/- 16.983093700744437)


In [10]:
test_predictions_tower.to_excel("test_predictions.xlsx", index=False)
test_location_estimates.to_excel("UTM_predictions.xlsx", index=False)